In [140]:
import torch
from random import randint
from utilities import chordUtil
from utilities import dataImport
from utilities.chordUtil import *
from utilities.dataImport import *
from sklearn.model_selection import train_test_split
import os
import numpy as np
import random
import sys
import torch.nn as nn
import time
import math
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

In [141]:
def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [142]:
# CUDA for PyTorch
use_cuda = torch.cuda.is_available()
print("use_cuda")
print(use_cuda)
#use_cuda = False
device = torch.device("cuda:0" if use_cuda else "cpu")

use_cuda
True


In [143]:
# Init
#lenSeq = 16
lenSeq = 48
alpha = 'a1'
rootname = "inputs/jazz_xlab/"
filenames = os.listdir(rootname)
#filenames.remove(".DS_Store")
dictChord, listChord = chordUtil.getDictChord(eval(alpha))
print(len(dictChord))
print(dictChord)

37
{'E:dim': 0, 'E:maj': 1, 'A:maj': 2, 'F:dim': 3, 'B:maj': 4, 'B:min': 5, 'C:min': 6, 'D:min': 7, 'F#:maj': 8, 'D:dim': 9, 'A:dim': 10, 'A#:maj': 11, 'A#:min': 12, 'B:dim': 13, 'C#:dim': 14, 'E:min': 15, 'F:maj': 16, 'G:dim': 17, 'G#:min': 18, 'C#:min': 19, 'C:dim': 20, 'F:min': 21, 'D#:maj': 22, 'F#:min': 23, 'G:maj': 24, 'G#:dim': 25, 'F#:dim': 26, 'G:min': 27, 'D#:dim': 28, 'C:maj': 29, 'A#:dim': 30, 'C#:maj': 31, 'G#:maj': 32, 'D:maj': 33, 'D#:min': 34, 'A:min': 35, 'N': 36}


In [144]:
# Create datasets
files_train ,files_test = train_test_split(filenames,test_size=0.7)
dataset_train = dataImport.ChordSeqDataset(files_train, rootname, alpha, dictChord, lenSeq)
dataset_test = dataImport.ChordSeqDataset(files_test, rootname, alpha, dictChord, lenSeq)


In [145]:
# Create generators
params = {'batch_size': 64,
          'shuffle': True,
          'num_workers': 6}
training_generator = data.DataLoader(dataset_train, **params)
testing_generator = data.DataLoader(dataset_test, **params)


In [146]:
def train(local_batch, local_labels):

    optimizer.zero_grad()
    loss = 0

    local_labels = torch.tensor([torch.argmax(local_label) for local_label in local_labels])
    local_batch, local_labels = local_batch.to(device), local_labels.to(device)    
    
    output, (hidden, cell_state) = lstm_nn(local_batch)

    output = output[:,-1,:]
    loss = criterion(output, local_labels)
    loss.backward()
    optimizer.step()


    return output, loss.item() / len(local_batch)

In [147]:
def test(local_batch, local_labels):
    loss = 0

    local_labels = torch.tensor([torch.argmax(local_label) for local_label in local_labels])
    local_batch, local_labels = local_batch.to(device), local_labels.to(device)
    
    output, (hidden, cell_state) = lstm_nn(local_batch)

    output = output[:,-1,:]
    loss = criterion(output, local_labels)

    return output, loss.item() / len(local_batch)
    

In [ ]:
lstm_nn = nn.LSTM(len(dictChord), len(dictChord), num_layers = 15, batch_first = True, dropout = 0.1).to(device=device)

all_losses = []
test_losses = []
total_loss = 0 # Reset every plot_every iters
test_loss = 0
start = time.time()

#optimizer = torch.optim.Adam(lstm_nn.parameters(), lr = 1e-4)
optimizer = torch.optim.SGD(lstm_nn.parameters(), lr = 5e-2, momentum=0.9)


criterion = nn.NLLLoss()
#criterion = nn.CrossEntropyLoss()


print_every = 5
plot_every = 5
max_epochs = 300


# Begin training

for epoch in range(1, max_epochs):
    # Training
    for local_batch, local_labels in training_generator:
        # Transfer to GPU (nope later...)
        #local_batch, local_labels = local_batch.to(device), local_labels.to(device)
        
        output, loss = train(local_batch, local_labels)
        #print(loss)
        total_loss += loss

    if epoch % print_every == 0:
        print('train : %s (%d %d%%) %.4f' % (timeSince(start), epoch, epoch / max_epochs * 100, loss))

    if epoch % plot_every == 0:
        all_losses.append(total_loss / (plot_every ))
        total_loss = 0
        

    # Testing
    for local_batch, local_labels in testing_generator:
        # Transfer to GPU (nope later...)
        #local_batch, local_labels = local_batch.to(device), local_labels.to(device)
        output, loss = test(local_batch, local_labels)
        test_loss +=loss

    if epoch % print_every == 0:
        print('test : %s (%d %d%%) %.4f' % (timeSince(start), epoch, epoch / max_epochs * 100, loss))

    if epoch % plot_every == 0:
        test_losses.append(test_loss / (plot_every ))
        test_loss = 0



train : 0m 9s (5 1%) -0.0417
test : 0m 10s (5 1%) -0.0990
train : 0m 19s (10 3%) -0.0473
test : 0m 21s (10 3%) -0.0998
train : 0m 30s (15 5%) -0.0468
test : 0m 31s (15 5%) -0.1107
train : 0m 41s (20 6%) -0.0470
test : 0m 42s (20 6%) -0.1096
train : 0m 51s (25 8%) -0.0475
test : 0m 52s (25 8%) -0.1108
train : 1m 2s (30 10%) -0.0448
test : 1m 3s (30 10%) -0.1109
train : 1m 12s (35 11%) -0.0474
test : 1m 13s (35 11%) -0.1109
train : 1m 23s (40 13%) -0.0436
test : 1m 24s (40 13%) -0.1099
train : 1m 33s (45 15%) -0.0475
test : 1m 35s (45 15%) -0.1110
train : 1m 44s (50 16%) -0.0476
test : 1m 45s (50 16%) -0.1109
train : 1m 54s (55 18%) -0.0475
test : 1m 55s (55 18%) -0.1108
train : 2m 5s (60 20%) -0.0475
test : 2m 6s (60 20%) -0.1110
train : 2m 16s (65 21%) -0.0475
test : 2m 17s (65 21%) -0.1108
train : 2m 26s (70 23%) -0.0475
test : 2m 27s (70 23%) -0.1109
train : 2m 37s (75 25%) -0.0476
test : 2m 38s (75 25%) -0.1107
train : 2m 47s (80 26%) -0.0475
test : 2m 48s (80 26%) -0.1109
train : 2

In [ ]:
plt.figure()
plt.plot(all_losses, label="train")
plt.legend(loc='upper right', frameon=False)
#plt.show()
plt.plot(test_losses, label="test")
plt.legend(loc='upper right', frameon=False)
plt.show()

In [ ]:
#test_sequence = ["C:maj","F:maj","C:maj","C:maj","F:maj","F:maj","C:maj","C:maj","G:maj","F:maj","C:maj","G:maj","C:maj","C:maj","F:maj","C:maj"]

test_sequence = ["C:maj","C:maj","C:maj","C:maj","F:maj","F:maj","F:maj","F:maj",
                "C:maj","C:maj","C:maj","C:maj","C:maj","C:maj","C:maj","C:maj",
                "F:maj","F:maj","F:maj","F:maj","F:maj","F:maj","F:maj","F:maj",
                "C:maj","C:maj","C:maj","C:maj","C:maj","C:maj","C:maj","C:maj",
                "G:maj","G:maj","G:maj","G:maj","F:maj","F:maj","F:maj","F:maj",
                "C:maj","C:maj","C:maj","C:maj","G:maj","G:maj","G:maj","G:maj",]

test_sequence_tensor = torch.zeros(1, len(test_sequence), len(dictChord)).to(device)
for t in range(len(test_sequence)):
    test_sequence_tensor[0, t, dictChord[test_sequence[t]]] = 1



In [ ]:
generation_lenght = 120

generated_sequence = [0 for i in range(generation_lenght)]
generated_sequence[0:lenSeq] = test_sequence



for t in range(generation_lenght-lenSeq):
    if t == 0:
        output, (hidden, cell_state) = lstm_nn(test_sequence_tensor)
        generated_sequence[t+lenSeq] = listChord[torch.argmax(output[0,-1]).item()]
        
    else : 
        last_chords_output = torch.zeros(1, lenSeq, len(dictChord)).to(device)
        for i in range(lenSeq):
            last_chords_output[0, i, torch.argmax(output[0][-1]).item()] = 1
        #last_chords_output = torch.tensor([[torch.argmax(output[0][i]).item() for i in range(lenSeq)]])
        last_chords_output.to(device)

        
        output, (hidden, cell_state) = lstm_nn(last_chords_output, (hidden, cell_state))
        generated_sequence[t+lenSeq] = listChord[torch.argmax(output[0][-1]).item()]

        
for i in range(generation_lenght):
    if i%4 == 0:
        print(generated_sequence[i:i+4])
    if i == lenSeq-1 :
        print("generated :")